In [41]:
import numpy as np 
import pandas as pd


In [42]:
data = pd.read_csv('ballons.csv')
target = data.columns[-1]
print(target)

def prior_probability(df, target):
    class_count = df[target].value_counts()
    prior = class_count / len(df)
    return prior

def training(df, className):
    feature_prob_given_class = []
    features = df.columns[:-1]
    class_counts = df[className].value_counts()

    for feature in features:
        class_feature_counts = df.groupby([className, feature]).size().unstack()
        prob_given_class = class_feature_counts.div(class_counts, axis=0)
        feature_prob_given_class.append(prob_given_class)
        
    return feature_prob_given_class

def naive_bayes_classifer(df, X_test, Y_test):
    class_values = df[Y_test].unique()
    feature_prob_given_class = training(df, Y_test)
    prior = prior_probability(df, Y_test)

    predictions = []
    X_test = pd.DataFrame(X_test)
    for i, test_case in X_test.iterrows():
        posterior_class = prior.copy()
        for j, feature in enumerate(test_case):
            for k, c in enumerate(class_values):
                try:
                    posterior_class[k] *= feature_prob_given_class[j].loc[c, feature]
                except KeyError:
                    posterior_class[k] = 0
        predictions.append(class_values[posterior_class.idxmax()])
    return predictions


Inflated


In [43]:
# for ballons Dataset
data.replace('SMALL', 0, inplace=True)
data.replace('LARGE', 1, inplace=True)
data.replace('PURPLE', 0, inplace=True)
data.replace('YELLOW', 1, inplace=True)
data.replace('DIP', 0, inplace=True)
data.replace('STRETCH', 1, inplace=True)
data.replace('CHILD', 0, inplace=True)
data.replace('ADULT', 1, inplace=True)
data.replace('T', 1, inplace=True)
data.replace('F', 0, inplace=True)

from sklearn.model_selection import train_test_split
train, test = train_test_split(data, test_size=.4)

X_test = test.iloc[:,:-1].values
Y_test = test.iloc[:,-1].values 
Y_pred = naive_bayes_classifer(data, X_test=X_test, Y_test=target)
print("Predicted: ",Y_pred, "\nActual: ", Y_test)

Predicted:  [0, 1, 0, 1, 0, 0, 0, 0] 
Actual:  [0 1 0 1 0 0 0 0]


In [44]:
from sklearn.metrics import confusion_matrix, f1_score , accuracy_score
print("Confusion Matrix: \n", confusion_matrix(Y_test, Y_pred))
print("Accuracy: ",accuracy_score(Y_test, Y_pred)*100, "%")

Confusion Matrix: 
 [[6 0]
 [0 2]]
Accuracy:  100.0 %


In [45]:
from sklearn.naive_bayes import GaussianNB

model = GaussianNB()
model.fit(train.iloc[:,:-1], train.iloc[:,-1])
Y_pred2 = model.predict(test.iloc[:,:-1])

print("Confusion Matrix: \n", confusion_matrix(Y_test, Y_pred2))
print("Accuracy: ",accuracy_score(Y_test, Y_pred2)*100, "%")

Confusion Matrix: 
 [[6 0]
 [0 2]]
Accuracy:  100.0 %
